In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
import pandas as pd
# pd.set_option('display.max_columns', None) # 데이터프레임 컬럼 display

import warnings
warnings.filterwarnings('ignore') # 경고 무시

In [3]:
def get_result(driver) :
    #--------------------- table 태그 읽기 - 팀 별 1 ~ 5 세트 결과 ---------------------#
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    table = soup.find_all('table')
    set_result = pd.read_html(str(table))[1] # 1 ~ 5 세트 별 득점 결과

    #--------------------- 날짜 정보 읽기 ---------------------#
    date_selector = '#wrp_content > article.wrp_recentgame.wrp_result > table > thead > tr > th'
    date = soup.select_one(date_selector).get_text()
    date = date.replace('\xa0', '').split('/')[0].strip().split()
    date = pd.to_datetime(date[0][:-1] + '/' + date[1][:-1] + '/' + date[2][:-1])

    #
    # 첫 번째 팀 정보 읽기
    #

    #--------------------- 버튼 클릭 - 선수 기록 ---------------------#
    x_path = '//*[@id="wrp_content"]/article[2]/ul/li[2]/a/span' # 선수 기록 버튼
    cursor = driver.find_element(By.XPATH,x_path)
    cursor.click()
    time.sleep(1)

    #--------------------- table 태그 읽기 - 선수 별 출전 세트, 득점, 공격 종합 ---------------------#
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    table = soup.find_all('table')

    team1 = pd.read_html(str(table))[3] # 명단 table
    team1_info = pd.read_html(str(table))[4] # 출전 세트, 득점, 공격 종합 table
    team1_info.columns = ['_'.join(col).strip() for col in team1_info.columns]
    team1 = pd.concat([team1, team1_info], axis = 1)

    #--------------------- 나머지 기록 읽기 ---------------------#
    for _ in range(4) :
        #--------------------- 버튼 클릭 - 다음 기록 ---------------------#
        x_path = '//*[@id="team1"]/div/div[1]/a[2]' # 다음 기록 버튼
        cursor = driver.find_element(By.XPATH,x_path)
        cursor.click()
        time.sleep(1)


        #--------------------- table 태그 읽기 - 다음 기록들 ---------------------#
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        table = soup.find_all('table')

        team1_info = pd.read_html(str(table))[4] # 다음 기록
        team1_info.columns = ['_'.join(col).strip() for col in team1_info.columns]
        team1 = pd.concat([team1, team1_info], axis = 1)


    #
    # 두 번째 팀 정보 읽기
    #

    #--------------------- 버튼 클릭 - 두 번째 팀 ---------------------#
    try :
        x_path = '//*[@id="tab2"]/div[4]/ul/li[2]/a/span' # 두 번째 팀 버튼
        cursor = driver.find_element(By.XPATH,x_path)
    except :
        try :
            x_path = '//*[@id="tab2"]/div[2]/ul/li[2]/a/span' # 두 번째 팀 버튼
            cursor = driver.find_element(By.XPATH,x_path)
        except :
            try :
                x_path = '//*[@id="tab2"]/div[3]/ul/li[2]/a/span'
                cursor = driver.find_element(By.XPATH,x_path)
            except:
                x_path = '//*[@id="tab2"]/div[1]/ul/li[2]/a/span' # 두 번째 팀 버튼
                cursor = driver.find_element(By.XPATH,x_path)
    cursor.click()
    time.sleep(1)

    #--------------------- table 태그 읽기 - 선수 별 출전 세트, 득점, 공격 종합 ---------------------#
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    table = soup.find_all('table')

    team2 = pd.read_html(str(table))[5] # 명단 table
    team2_info = pd.read_html(str(table))[6] # 출전 세트, 득점, 공격 종합 table
    team2_info.columns = ['_'.join(col).strip() for col in team2_info.columns]
    team2 = pd.concat([team2, team2_info], axis = 1)

    #--------------------- 나머지 기록 읽기 ---------------------#
    for _ in range(4) :
        #--------------------- 버튼 클릭 - 다음 기록 ---------------------#
        x_path = '//*[@id="team2"]/div/div[1]/a[2]' # 다음 기록 버튼
        cursor = driver.find_element(By.XPATH,x_path)
        cursor.click()
        time.sleep(1)


        #--------------------- table 태그 읽기 - 다음 기록들 ---------------------#
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        table = soup.find_all('table')

        team2_info = pd.read_html(str(table))[6] # 다음 기록
        team2_info.columns = ['_'.join(col).strip() for col in team2_info.columns]
        team2 = pd.concat([team2, team2_info], axis = 1)
    
    return set_result, date, team1, team2

In [4]:
def clean_result(set_result, date, team1, team2) :
    # No. 열 삭제
    team1 = team1.drop('No.', axis = 1)
    team2 = team2.drop('No.', axis = 1)

    
    # 3경기 이상 출전한 선수만 유지 (출전 세트 결측치가 3개 이상인 선수 제거)
    subset = ['출전세트_1set', '출전세트_2set', '출전세트_3set', '출전세트_4set', '출전세트_5set']

    # thresh = 3 : 정상 값(Not NaN)이 3개 이상만 유지
    team1 = team1.dropna(subset = subset, thresh = 3)
    team2 = team2.dropna(subset = subset, thresh = 3)

    # 스타팅멤버 표시 (3세트 이상 스타팅멤버라면 1 아니면 0)
    team1['스타팅멤버'] = 0
    team2['스타팅멤버'] = 0

    team1.loc[(team1.iloc[:, 1:6] == 'O').sum(axis = 1) >= 3, '스타팅멤버'] = 1
    team2.loc[(team2.iloc[:, 1:6] == 'O').sum(axis = 1) >= 3, '스타팅멤버'] = 1
    
    
    # 출전 세트 열 삭제
    subset = ['출전세트_1set', '출전세트_2set', '출전세트_3set', '출전세트_4set', '출전세트_5set']

    team1 = team1.drop(subset, axis = 1)
    team2 = team2.drop(subset, axis = 1)

    
    # 이름과 포지션 분리하기
    # "이름" 열 : 이름 정보
    # "포지션" 열 : 포지션 정보
    team1['포지션'] = team1['이름'].str.split().str[1].str.strip('()')
    team1['이름'] = team1['이름'].str.split().str[0]

    team2['포지션'] = team2['이름'].str.split().str[1].str.strip('()')
    team2['이름'] = team2['이름'].str.split().str[0]

    
    # 선수 별 득점 점유율 구하기 (개인 별 득점을 팀 전체의 득점으로 나누기)
    team1_total_score = team1['득점_득점'].sum()
    team2_total_score = team2['득점_득점'].sum()
    team1['득점점유율'] = team1['득점_득점'] / team1_total_score
    team2['득점점유율'] = team2['득점_득점'] / team2_total_score
    
    
    # 컬럼 선택
    columns = [
        '이름',
        '득점_득점',
        '공격종합_시도',
        '공격종합_성공',
        '공격종합_공격차단',
        '공격종합_범실',
        '공격종합_성공률',
        '공격종합_점유율',
        '오픈_시도',
        '오픈_성공',
        '오픈_공격차단',
        '오픈_범실',
        '오픈_성공률',
        '오픈_점유율',
        '시간차_시도',
        '시간차_성공',
        '시간차_공격차단',
        '시간차_범실',
        '시간차_성공률',
        '시간차_점유율',
        '이동_시도',
        '이동_성공',
        '이동_공격차단',
        '이동_범실',
        '이동_성공률',
        '이동_점유율',
        '후위_시도',
        '후위_성공',
        '후위_공격차단',
        '후위_범실',
        '후위_성공률',
        '후위_점유율',
        '속공_시도',
        '속공_성공',
        '속공_공격차단',
        '속공_범실',
        '속공_성공률',
        '속공_점유율',
        '퀵오픈_시도',
        '퀵오픈_성공',
        '퀵오픈_공격차단',
        '퀵오픈_범실',
        '퀵오픈_성공률',
        '퀵오픈_점유율',
        '서브_시도',
        '서브_성공',
        '서브_범실',
        '서브_성공률',
        '서브_점유율',
        '디그_시도',
        '디그_성공',
        '디그_실패',
        '디그_범실',
        '디그_세트당',
        '디그_점유율',
        '세트_시도',
        '세트_성공',
        '세트_범실',
        '세트_세트당',
        '세트_점유율',
        '리시브_시도',
        '리시브_정확',
        '리시브_실패',
        '리시브_세트당',
        '리시브_점유율',
        '블로킹_시도',
        '블로킹_성공',
        '블로킹_유효블락',
        '블로킹_실패',
        '블로킹_범실',
        '블로킹_세트당',
        '블로킹_점유율',
        '블로킹_어시스트',
        '벌칙_벌칙',
        '범실_범실',
        '포지션',
        '득점점유율',
        '스타팅멤버'
    ]
    team1 = team1[columns]
    team2 = team2[columns]
    
    
    # 경기날짜, 팀명, 결과 정보 추가
    team1['팀명'] = set_result.iloc[0,0]
    team2['팀명'] = set_result.iloc[1,0]
    team1['결과'] = 1 if int(set_result.iloc[0,-1]) > int(set_result.iloc[1,-1]) else 0
    team2['결과'] = 0 if int(set_result.iloc[0,-1]) > int(set_result.iloc[1,-1]) else 1
    team1['경기날짜'] = date
    team2['경기날짜'] = date
    team1['경기장'] = 1
    team2['경기장'] = 0

    # 컬럼 순서 변경
    team1 = team1[['팀명', '경기날짜', '경기장' ,'결과'] + columns]
    team2 = team2[['팀명', '경기날짜', '경기장', '결과'] + columns]
    
    
    # 연결
    team = pd.concat([team1, team2])
    team
    
    return team

In [5]:
seasons = [f'0{num}' for num in range(14, 19)]
yymms = [
pd.date_range('2017-10', '2018-04', freq = 'M').strftime('%Y-%m').to_list(),
pd.date_range('2018-10', '2019-04', freq = 'M').strftime('%Y-%m').to_list(),
pd.date_range('2019-10', '2020-04', freq = 'M').strftime('%Y-%m').to_list(),
pd.date_range('2020-10', '2021-05', freq = 'M').strftime('%Y-%m').to_list(),
pd.date_range('2021-10', '2022-03', freq = 'M').strftime('%Y-%m').to_list()
]

In [208]:
dfs = []
driver = webdriver.Chrome()

for season, dates in zip(seasons, yymms) :
    for date in dates :
        # 경기 스케줄 url 접속
        url = f'https://www.kovo.co.kr/game/v-league/11110_schedule_list.asp?season={season}&team=&yymm={date}&r_round='
        driver.get(url)
        time.sleep(1)


        # 경기 스케줄을 데이터프레임으로 저장 후 여자 경기의 인덱스를 idx에 저장
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        table = soup.find_all('table')
        try :
            df = pd.read_html(str(table))[0]
            idx = df[df['구분'] == '남자'].index.to_list()
        except :
            df = pd.read_html(str(table))[2]
            idx = df[df['구분'] == '남자'].index.to_list()
        idx = [i + 1 for i in idx]

        

        # 각 경기의 상세결과 버튼 클릭
        for i in idx :
            # 상세결과 버튼 클릭
            try :
                x_path = f'//*[@id="type1"]/div/table/tbody/tr[{i}]/td[10]/a[3]' # 상세결과 버튼
                cursor = driver.find_element(By.XPATH,x_path)
            except :
                x_path = f'//*[@id="type1"]/div/table/tbody/tr[{i}]/td[10]/a[2]' # 상세결과 버튼
                cursor = driver.find_element(By.XPATH,x_path)
            cursor.click()
            time.sleep(1)

            # 함수 적용
            set_result, date, team1, team2 = get_result(driver)
            team = clean_result(set_result, date, team1, team2)
            dfs.append(team)

            # 다시 경기 스케줄 url로 이동
            driver.get(url)
            time.sleep(1)
df = pd.concat(dfs)
df

AttributeError: 'DataFrame' object has no attribute 'sleep'

In [50]:
df.to_csv('남자배구.csv', index = False)

# 남자배구 세트당 득점 및 경기시간

## 17~22까지 세트별 점수 및 경기시간

In [6]:
seasons = [f'0{num}' for num in range(14, 19)]
yymms = [
pd.date_range('2017-10', '2018-04', freq = 'M').strftime('%Y-%m').to_list(),
pd.date_range('2018-10', '2019-04', freq = 'M').strftime('%Y-%m').to_list(),
pd.date_range('2019-10', '2020-04', freq = 'M').strftime('%Y-%m').to_list(),
pd.date_range('2020-10', '2021-05', freq = 'M').strftime('%Y-%m').to_list(),
pd.date_range('2021-10', '2022-03', freq = 'M').strftime('%Y-%m').to_list()
]

driver = webdriver.Chrome()
dfs=[]
for season, dates in zip(seasons, yymms) :
    for date in dates :
        # 경기 스케줄 url 접속
        url = f'https://www.kovo.co.kr/game/v-league/11110_schedule_list.asp?season={season}&team=&yymm={date}&r_round='
        driver.get(url)
        time.sleep(1)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        table = soup.find_all('table')
        df = pd.read_html(str(table))[0]
        # '남자', '여자' 만 바꾸면 됨
        idx = df[df['구분'] == '남자'].index.to_list()
        idx = [i + 1 for i in idx]

        # 각 경기의 상세결과 버튼 클릭
        for i in idx :
            # 상세결과 버튼 클릭
            try :
                x_path = f'//*[@id="type1"]/div/table/tbody/tr[{i}]/td[10]/a[3]' # 상세결과 버튼
                cursor = driver.find_element(By.XPATH,x_path)
            except :
                x_path = f'//*[@id="type1"]/div/table/tbody/tr[{i}]/td[10]/a[2]' # 상세결과 버튼
                cursor = driver.find_element(By.XPATH,x_path)
            cursor.click()
            time.sleep(1)
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            table = soup.find_all('table')
            set_result = pd.read_html(str(table))[1]
            dfs.append([set_result])

            # 다시 경기 스케줄 url로 이동
            driver.get(url)
            time.sleep(1)

In [83]:
len(dfs)

610

In [75]:
result = pd.DataFrame(dfs[0][0]).iloc[:2,:]
result.columns=['팀명','1SET','2SET','3SET','4SET','5SET','TOTAL']
array = pd.DataFrame(dfs[0][0]).iloc[2,:].str[:-1]
array = array.to_frame().T.drop(['TOTAL'],axis=1)
array.columns=['팀명','경기시간_1SET','경기시간_2SET','경기시간_3SET','경기시간_4SET','경기시간_5SET']
array = pd.concat([array,array])
array.reset_index(drop =True,inplace=True )
total = pd.concat([result,array.iloc[:,1:]],axis=1)

In [76]:
for i in range(1,len(dfs)):
    result = pd.DataFrame(dfs[i][0]).iloc[:2,:]
    result.columns=['팀명','1SET','2SET','3SET','4SET','5SET','TOTAL']
    array = pd.DataFrame(dfs[i][0]).iloc[2,:].str[:-1]
    array = array.to_frame().T.drop(['TOTAL'],axis=1)
    array.columns=['팀명','경기시간_1SET','경기시간_2SET','경기시간_3SET','경기시간_4SET','경기시간_5SET']
    array = pd.concat([array,array])
    array.reset_index(drop =True,inplace=True )
    add = pd.concat([result,array.iloc[:,1:]],axis=1)
    total = pd.concat([total,add])
total.iloc[:,1:] = total.iloc[:,1:].astype('int32')
total

,팀명,1SET,2SET,3SET,4SET,5SET,TOTAL,경기시간_1SET,경기시간_2SET,경기시간_3SET,경기시간_4SET,경기시간_5SET
0,현대캐피탈,21,25,25,33,0,104,27,30,30,37,0
1,대한항공,25,23,21,31,0,100,27,30,30,37,0
0,KB손해보험,18,25,25,23,15,106,23,30,23,27,17
1,삼성화재,25,22,18,25,13,103,23,30,23,27,17
0,OK저축은행,21,25,19,25,15,105,31,28,27,25,16
...,...,...,...,...,...,...,...,...,...,...,...,...
1,우리카드,23,25,20,17,0,85,29,28,29,25,0
0,한국전력,25,25,19,22,15,106,29,30,27,28,19
1,KB손해보험,22,22,25,25,10,104,29,30,27,28,19
0,현대캐피탈,20,25,25,0,0,70,27,36,42,0,0


In [77]:
import numpy as np

In [78]:
G_num = total.shape[0] // 2
total['경기번호'] = np.array([[i,i] for i in range(1, G_num+1)]).flatten()
total.reset_index(drop=True,inplace=True)
total

,팀명,1SET,2SET,3SET,4SET,5SET,TOTAL,경기시간_1SET,경기시간_2SET,경기시간_3SET,경기시간_4SET,경기시간_5SET,경기번호
0,현대캐피탈,21,25,25,33,0,104,27,30,30,37,0,1
1,대한항공,25,23,21,31,0,100,27,30,30,37,0,1
2,KB손해보험,18,25,25,23,15,106,23,30,23,27,17,2
3,삼성화재,25,22,18,25,13,103,23,30,23,27,17,2
4,OK저축은행,21,25,19,25,15,105,31,28,27,25,16,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1215,우리카드,23,25,20,17,0,85,29,28,29,25,0,608
1216,한국전력,25,25,19,22,15,106,29,30,27,28,19,609
1217,KB손해보험,22,22,25,25,10,104,29,30,27,28,19,609
1218,현대캐피탈,20,25,25,0,0,70,27,36,42,0,0,610


In [80]:
남자배구_시간 = total[['경기번호','팀명']]
남자배구_시간

,경기번호,팀명,1SET,2SET,3SET,4SET,5SET,TOTAL,경기시간_1SET,경기시간_2SET,경기시간_3SET,경기시간_4SET,경기시간_5SET
0,1,현대캐피탈,21,25,25,33,0,104,27,30,30,37,0
1,1,대한항공,25,23,21,31,0,100,27,30,30,37,0
2,2,KB손해보험,18,25,25,23,15,106,23,30,23,27,17
3,2,삼성화재,25,22,18,25,13,103,23,30,23,27,17
4,3,OK저축은행,21,25,19,25,15,105,31,28,27,25,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1215,608,우리카드,23,25,20,17,0,85,29,28,29,25,0
1216,609,한국전력,25,25,19,22,15,106,29,30,27,28,19
1217,609,KB손해보험,22,22,25,25,10,104,29,30,27,28,19
1218,610,현대캐피탈,20,25,25,0,0,70,27,36,42,0,0


In [81]:
남자배구_시간.to_csv('남자배구_시간.csv')

In [25]:
result = pd.DataFrame(dfs[0][0]).iloc[:2,:]
result.columns=['팀명','1SET','2SET','3SET','4SET','5SET','TOTAL']
array = pd.DataFrame(dfs[0][0]).iloc[2,:-1].str[:-1]
array
# array = array.to_frame().T.drop(['TOTAL'],axis=1)
# array = pd.concat([array,array])
# array.reset_index(drop =True,inplace=True)
# array.iloc[:,1:]

남자부     경기시
1SET    27 
2SET    30 
3SET    30 
4SET    37 
5SET     0 
Name: 2, dtype: object

In [85]:
남자배구 = pd.read_csv('남자배구.csv',encoding='utf8')
남자배구

,팀명,경기날짜,경기장,결과,이름,득점_득점,공격종합_시도,공격종합_성공,공격종합_공격차단,공격종합_범실,공격종합_성공률,공격종합_점유율,오픈_시도,오픈_성공,오픈_공격차단,오픈_범실,오픈_성공률,오픈_점유율,시간차_시도,시간차_성공,시간차_공격차단,시간차_범실,시간차_성공률,시간차_점유율,이동_시도,이동_성공,이동_공격차단,이동_범실,이동_성공률,이동_점유율,후위_시도,후위_성공,후위_공격차단,후위_범실,후위_성공률,후위_점유율,속공_시도,속공_성공,속공_공격차단,속공_범실,속공_성공률,속공_점유율,퀵오픈_시도,퀵오픈_성공,퀵오픈_공격차단,퀵오픈_범실,퀵오픈_성공률,퀵오픈_점유율,서브_시도,서브_성공,서브_범실,서브_성공률,서브_점유율,디그_시도,디그_성공,디그_실패,디그_범실,디그_세트당,디그_점유율,세트_시도,세트_성공,세트_범실,세트_세트당,세트_점유율,리시브_시도,리시브_정확,리시브_실패,리시브_세트당,리시브_점유율,블로킹_시도,블로킹_성공,블로킹_유효블락,블로킹_실패,블로킹_범실,블로킹_세트당,블로킹_점유율,블로킹_어시스트,벌칙_벌칙,범실_범실,포지션,득점점유율,스타팅멤버
0,현대캐피탈,2017-10-14,1,1,안드레아스,23,34,21,0,1,61.76,35.05,10,6,0,1,60.00,34.48,1,1,0,0,100.00,20.0,0,0,0,0,0.0,0.0,8,6,0,0,75.00,38.1,0,0,0,0,0.00,0.00,15,8,0,0,53.33,53.57,14,0,8,0.00,13.59,6,4,2,0,1.00,9.68,2,0,0,0.00,1.92,17,10,0,2.50,22.08,12,2,4,2,1,0.50,25.00,3,0,10,L,0.338235,1
1,현대캐피탈,2017-10-14,1,1,문성민,18,32,17,3,2,53.13,32.99,11,5,1,1,45.45,37.93,1,1,0,0,100.00,20.0,0,0,0,0,0.0,0.0,13,6,2,1,46.15,61.9,0,0,0,0,0.00,0.00,7,5,0,0,71.43,25.00,15,1,5,0.25,14.56,8,5,3,0,1.25,12.90,2,1,0,0.25,1.92,0,0,0,0.00,0.00,7,0,2,4,0,0.00,14.58,0,0,7,R,0.264706,1
2,현대캐피탈,2017-10-14,1,1,신영석,10,8,4,1,0,50.00,8.25,2,1,0,0,50.00,6.90,0,0,0,0,0.00,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.00,0.0,6,3,1,0,50.00,42.86,0,0,0,0,0.00,0.00,16,1,0,0.25,15.53,5,5,0,0,1.25,8.06,4,1,1,0.25,3.85,2,1,0,0.25,2.60,11,5,0,4,0,1.25,22.92,2,0,1,C,0.147059,1
3,현대캐피탈,2017-10-14,1,1,김재휘,7,8,4,0,0,50.00,8.25,0,0,0,0,0.00,0.00,0,0,0,0,0.00,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.00,0.0,8,4,0,0,50.00,57.14,0,0,0,0,0.00,0.00,14,1,4,0.25,13.59,3,2,1,0,0.50,4.84,1,1,0,0.25,0.96,0,0,0,0.00,0.00,10,2,2,4,1,0.50,20.83,0,0,5,C,0.102941,1
4,현대캐피탈,2017-10-14,1,1,이시우,7,8,6,0,1,75.00,8.25,1,0,0,1,0.00,3.45,3,2,0,0,66.67,60.0,0,0,0,0,0.0,0.0,0,0,0,0,0.00,0.0,0,0,0,0,0.00,0.00,4,4,0,0,100.00,14.29,10,1,3,0.25,9.71,5,3,2,0,0.75,8.06,3,2,0,0.50,2.88,15,7,2,1.25,19.48,1,0,0,1,0,0.00,2.08,1,0,4,L,0.102941,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11014,OK금융그룹,2022-02-13,0,1,박원빈,1,5,0,0,1,0.00,6.02,0,0,0,0,0.00,0.00,0,0,0,0,0.00,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.00,0.0,5,0,0,1,0.00,55.56,0,0,0,0,0.00,0.00,8,0,0,0.00,10.39,4,3,1,0,1.00,10.53,5,1,0,0.33,5.95,0,0,0,0.00,0.00,13,1,5,4,0,0.33,30.95,2,0,1,C,0.019608,1
11015,OK금융그룹,2022-02-13,0,1,진상헌,1,3,1,0,0,33.33,3.61,0,0,0,0,0.00,0.00,0,0,0,0,0.00,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.00,0.0,3,1,0,0,33.33,33.33,0,0,0,0,0.00,0.00,6,0,0,0.00,7.79,0,0,0,0,0.00,0.00,0,0,0,0.00,0.00,0,0,0,0.00,0.00,7,0,1,4,0,0.00,16.67,1,0,0,C,0.019608,1
11016,OK금융그룹,2022-02-13,0,1,전병선,1,0,0,0,0,0.00,0.00,0,0,0,0,0.00,0.00,0,0,0,0,0.00,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.00,0.0,0,0,0,0,0.00,0.00,0,0,0,0,0.00,0.00,5,1,0,0.33,6.49,1,0,1,0,0.00,2.63,0,0,0,0.00,0.00,0,0,0,0.00,0.00,0,0,0,0,0,0.00,0.00,0,0,0,R,0.019608,0
11017,OK금융그룹,2022-02-13,0,1,부용찬,0,0,0,0,0,0.00,0.00,0,0,0,0,0.00,0.00,0,0,0,0,0.00,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.00,0.0,0,0,0,0,0.00,0.00,0,0,0,0,0.00,0.00,0,0,0,0.00,0.00,8,7,1,0,2.33,21.05,0,0,0,0.00,0.00,0,0,0,0.00,0.00,0,0,0,0,0,0.00,0.00,0,0,0,Li,0.000000,0


# 여자배구 경기시간 크롤링

In [3]:
seasons = [f'0{num}' for num in range(14, 19)]
yymms = [
pd.date_range('2017-10', '2018-04', freq = 'M').strftime('%Y-%m').to_list(),
pd.date_range('2018-10', '2019-04', freq = 'M').strftime('%Y-%m').to_list(),
pd.date_range('2019-10', '2020-04', freq = 'M').strftime('%Y-%m').to_list(),
pd.date_range('2020-10', '2021-05', freq = 'M').strftime('%Y-%m').to_list(),
pd.date_range('2021-10', '2022-03', freq = 'M').strftime('%Y-%m').to_list()
]

driver = webdriver.Chrome()
dfs=[]
for season, dates in zip(seasons, yymms) :
    for date in dates :
        # 경기 스케줄 url 접속
        url = f'https://www.kovo.co.kr/game/v-league/11110_schedule_list.asp?season={season}&team=&yymm={date}&r_round='
        driver.get(url)
        time.sleep(1)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        table = soup.find_all('table')
        df = pd.read_html(str(table))[0]
        # '남자', '여자' 만 바꾸면 됨
        idx = df[df['구분'] == '여자'].index.to_list()
        idx = [i + 1 for i in idx]

        # 각 경기의 상세결과 버튼 클릭
        for i in idx :
            # 상세결과 버튼 클릭
            try :
                x_path = f'//*[@id="type1"]/div/table/tbody/tr[{i}]/td[10]/a[3]' # 상세결과 버튼
                cursor = driver.find_element(By.XPATH,x_path)
            except :
                x_path = f'//*[@id="type1"]/div/table/tbody/tr[{i}]/td[10]/a[2]' # 상세결과 버튼
                cursor = driver.find_element(By.XPATH,x_path)
            cursor.click()
            time.sleep(1)
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            table = soup.find_all('table')
            set_result = pd.read_html(str(table))[1]
            dfs.append([set_result])

            # 다시 경기 스케줄 url로 이동
            driver.get(url)
            time.sleep(1)

In [53]:
len(dfs)

474

In [76]:
result = pd.DataFrame(dfs[0][0]).iloc[:2,:]
result.columns=['팀명','1SET','2SET','3SET','4SET','5SET','TOTAL']
array = pd.DataFrame(dfs[0][0]).iloc[2,:].str[:-1]
array = array.to_frame().T.drop(['TOTAL'],axis=1)
array.columns=['팀명','경기시간_1SET','경기시간_2SET','경기시간_3SET','경기시간_4SET','경기시간_5SET']
array = pd.concat([array,array])
array.reset_index(drop =True,inplace=True )
total = pd.concat([result,array.iloc[:,1:]],axis=1)
for i in range(1,len(dfs)):
    result = pd.DataFrame(dfs[i][0]).iloc[:2,:]
    result.columns=['팀명','1SET','2SET','3SET','4SET','5SET','TOTAL']
    array = pd.DataFrame(dfs[i][0]).iloc[2,:].str[:-1]
    array = array.to_frame().T.drop(['TOTAL'],axis=1)
    array.columns=['팀명','경기시간_1SET','경기시간_2SET','경기시간_3SET','경기시간_4SET','경기시간_5SET']
    array = pd.concat([array,array])
    array.reset_index(drop =True,inplace=True )
    add = pd.concat([result,array.iloc[:,1:]],axis=1)
    total = pd.concat([total,add])
total.iloc[:,1:] = total.iloc[:,1:].astype('int32')
total

,팀명,1SET,2SET,3SET,4SET,5SET,TOTAL,경기시간_1SET,경기시간_2SET,경기시간_3SET,경기시간_4SET,경기시간_5SET
0,IBK기업은행,11,25,25,20,15,96,22,27,29,29,19
1,흥국생명,25,23,22,25,13,108,22,27,29,29,19
0,KGC인삼공사,25,23,25,15,20,108,32,28,28,23,25
1,현대건설,23,25,21,25,22,116,32,28,28,23,25
0,한국도로공사,23,21,26,25,12,107,29,29,30,29,20
...,...,...,...,...,...,...,...,...,...,...,...,...
1,흥국생명,17,22,14,0,0,53,26,30,24,0,0
0,한국도로공사,25,25,25,0,0,75,23,30,32,0,0
1,페퍼저축은행,12,20,23,0,0,55,23,30,32,0,0
0,GS칼텍스,25,28,25,0,0,78,23,33,23,0,0


## 여자배구 경기번호, 홈_어웨이 추가 

In [77]:
import numpy as np
G_num = total.shape[0] // 2
total['경기번호'] = np.array([[i,i] for i in range(1, G_num+1)]).flatten()
total = total.reset_index(drop=True).reset_index()
total['홈_어웨이'] = np.where(total['index'] % 2 == 0,1,0)
# total = total.groupby(['경기번호','팀명'])

,index,팀명,1SET,2SET,3SET,4SET,5SET,TOTAL,경기시간_1SET,경기시간_2SET,경기시간_3SET,경기시간_4SET,경기시간_5SET,경기번호,홈_어웨이
0,0,IBK기업은행,11,25,25,20,15,96,22,27,29,29,19,1,1
1,1,흥국생명,25,23,22,25,13,108,22,27,29,29,19,1,0
2,2,KGC인삼공사,25,23,25,15,20,108,32,28,28,23,25,2,1
3,3,현대건설,23,25,21,25,22,116,32,28,28,23,25,2,0
4,4,한국도로공사,23,21,26,25,12,107,29,29,30,29,20,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
943,943,흥국생명,17,22,14,0,0,53,26,30,24,0,0,472,0
944,944,한국도로공사,25,25,25,0,0,75,23,30,32,0,0,473,1
945,945,페퍼저축은행,12,20,23,0,0,55,23,30,32,0,0,473,0
946,946,GS칼텍스,25,28,25,0,0,78,23,33,23,0,0,474,1


In [80]:
total = total.groupby(['경기번호','팀명','홈_어웨이']).sum().reset_index()
# total.drop(columns=['경기번호','팀명'], inplace=True)

In [87]:
(total.iloc[:,0:3]).to_csv('여자배구_홈_어웨이.csv',index = False)

## 경기별_수치에 날짜 및 홈 어웨이 추가

In [106]:
시즌 = pd.read_csv('경기별 수치.csv',encoding='utf8')
시즌.drop(columns=['Unnamed: 0','index','홈_어웨이'],inplace=True)
홈_어웨이 = pd.read_csv('여자배구_홈_어웨이.csv',encoding='utf8')
홈_어웨이.drop(columns=['경기번호','팀명'],inplace=True)
경기별_수치_홈어웨이 = pd.concat([시즌,홈_어웨이],axis=1)
경기별_수치_홈어웨이

,경기번호,팀명,결과,No.,득점_득점,공격종합_시도,공격종합_성공,공격종합_공격차단,공격종합_범실,공격종합_성공률,...,블로킹_유효블락,블로킹_실패,블로킹_범실,블로킹_세트당,블로킹_점유율,블로킹_어시스트,벌칙_벌칙,범실_범실,팀코드,홈_어웨이
0,1,IBK기업은행,0,89,69,179,51,15,10,143.88,...,37,30,2,2.25,100.01,8,0,25,3,1
1,1,흥국생명,1,127,83,193,63,11,15,135.52,...,38,12,0,3.30,100.02,10,0,27,1,0
2,2,KGC인삼공사,0,94,79,205,67,20,7,345.82,...,31,22,2,1.40,99.99,5,0,14,4,1
3,2,현대건설,1,102,102,198,77,7,12,186.93,...,33,30,3,4.00,100.01,17,0,29,5,0
4,3,GS칼텍스,1,138,88,175,72,9,9,303.46,...,28,25,0,1.45,100.00,6,0,25,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
943,472,흥국생명,0,111,38,108,32,8,8,186.29,...,16,18,2,0.99,99.99,3,0,15,1,0
944,473,페퍼저축은행,0,116,44,123,36,10,7,239.40,...,16,13,0,1.33,100.01,3,0,12,6,0
945,473,한국도로공사,1,124,63,109,49,4,3,321.63,...,21,19,0,3.33,100.01,9,0,11,2,1
946,474,GS칼텍스,1,164,67,91,52,8,0,563.85,...,21,17,1,3.34,99.98,8,0,7,0,1


In [44]:
여자배구_날짜용 = pd.read_csv('volleyball_raw.csv',encoding='utf8')
여자배구_날짜용 = 여자배구_날짜용.groupby(['경기번호','경기날짜','팀명']).sum().reset_index()

In [108]:
여자배구_날짜용['경기날짜']

0      2017-10-14
1      2017-10-14
2      2017-10-15
3      2017-10-15
4      2017-10-17
          ...    
943    2022-02-26
944    2022-02-27
945    2022-02-27
946    2022-02-28
947    2022-02-28
Name: 경기날짜, Length: 948, dtype: object

In [110]:
경기별_수치_홈어웨이['경기날짜']=여자배구_날짜용['경기날짜']
경기별_수치_홈어웨이 = 경기별_수치_홈어웨이.groupby(['경기번호','경기날짜','팀명','홈_어웨이']).sum().reset_index()
경기별_수치_홈어웨이

,경기번호,경기날짜,팀명,홈_어웨이,결과,No.,득점_득점,공격종합_시도,공격종합_성공,공격종합_공격차단,...,블로킹_성공,블로킹_유효블락,블로킹_실패,블로킹_범실,블로킹_세트당,블로킹_점유율,블로킹_어시스트,벌칙_벌칙,범실_범실,팀코드
0,1,2017-10-14,IBK기업은행,1,0,89,69,179,51,15,...,11,37,30,2,2.25,100.01,8,0,25,3
1,1,2017-10-14,흥국생명,0,1,127,83,193,63,11,...,15,38,12,0,3.30,100.02,10,0,27,1
2,2,2017-10-15,KGC인삼공사,1,0,94,79,205,67,20,...,7,31,22,2,1.40,99.99,5,0,14,4
3,2,2017-10-15,현대건설,0,1,102,102,198,77,7,...,20,33,30,3,4.00,100.01,17,0,29,5
4,3,2017-10-17,GS칼텍스,0,1,138,88,175,72,9,...,7,28,25,0,1.45,100.00,6,0,25,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
943,472,2022-02-26,흥국생명,0,0,111,38,108,32,8,...,3,16,18,2,0.99,99.99,3,0,15,1
944,473,2022-02-27,페퍼저축은행,0,0,116,44,123,36,10,...,4,16,13,0,1.33,100.01,3,0,12,6
945,473,2022-02-27,한국도로공사,1,1,124,63,109,49,4,...,10,21,19,0,3.33,100.01,9,0,11,2
946,474,2022-02-28,GS칼텍스,1,1,164,67,91,52,8,...,10,21,17,1,3.34,99.98,8,0,7,0


## 상대팀 추가

In [114]:
경기별_수치_홈어웨이.reset_index(inplace=True)
home_index = 경기별_수치_홈어웨이[경기별_수치_홈어웨이['index']/2+1==경기별_수치_홈어웨이['경기번호']].index
away_index = 경기별_수치_홈어웨이[경기별_수치_홈어웨이['index']/2+1!=경기별_수치_홈어웨이['경기번호']].index
경기별_수치_홈어웨이['상대팀'] = 경기별_수치_홈어웨이['팀명']
경기별_수치_홈어웨이.loc[home_index,'상대팀']=경기별_수치_홈어웨이.loc[away_index,'팀명'].values
경기별_수치_홈어웨이.loc[away_index,'상대팀']=경기별_수치_홈어웨이.loc[home_index,'팀명'].values

,index,경기번호,경기날짜,팀명,홈_어웨이,결과,No.,득점_득점,공격종합_시도,공격종합_성공,...,블로킹_유효블락,블로킹_실패,블로킹_범실,블로킹_세트당,블로킹_점유율,블로킹_어시스트,벌칙_벌칙,범실_범실,팀코드,상대팀
0,0,1,2017-10-14,IBK기업은행,1,0,89,69,179,51,...,37,30,2,2.25,100.01,8,0,25,3,흥국생명
1,1,1,2017-10-14,흥국생명,0,1,127,83,193,63,...,38,12,0,3.30,100.02,10,0,27,1,IBK기업은행
2,2,2,2017-10-15,KGC인삼공사,1,0,94,79,205,67,...,31,22,2,1.40,99.99,5,0,14,4,현대건설
3,3,2,2017-10-15,현대건설,0,1,102,102,198,77,...,33,30,3,4.00,100.01,17,0,29,5,KGC인삼공사
4,4,3,2017-10-17,GS칼텍스,0,1,138,88,175,72,...,28,25,0,1.45,100.00,6,0,25,0,한국도로공사
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
943,943,472,2022-02-26,흥국생명,0,0,111,38,108,32,...,16,18,2,0.99,99.99,3,0,15,1,IBK기업은행
944,944,473,2022-02-27,페퍼저축은행,0,0,116,44,123,36,...,16,13,0,1.33,100.01,3,0,12,6,한국도로공사
945,945,473,2022-02-27,한국도로공사,1,1,124,63,109,49,...,21,19,0,3.33,100.01,9,0,11,2,페퍼저축은행
946,946,474,2022-02-28,GS칼텍스,1,1,164,67,91,52,...,21,17,1,3.34,99.98,8,0,7,0,KGC인삼공사


In [116]:
경기별_수치_홈어웨이.drop(columns='index',inplace=True)
경기별_수치_홈어웨이

,경기번호,경기날짜,팀명,홈_어웨이,결과,No.,득점_득점,공격종합_시도,공격종합_성공,공격종합_공격차단,...,블로킹_유효블락,블로킹_실패,블로킹_범실,블로킹_세트당,블로킹_점유율,블로킹_어시스트,벌칙_벌칙,범실_범실,팀코드,상대팀
0,1,2017-10-14,IBK기업은행,1,0,89,69,179,51,15,...,37,30,2,2.25,100.01,8,0,25,3,흥국생명
1,1,2017-10-14,흥국생명,0,1,127,83,193,63,11,...,38,12,0,3.30,100.02,10,0,27,1,IBK기업은행
2,2,2017-10-15,KGC인삼공사,1,0,94,79,205,67,20,...,31,22,2,1.40,99.99,5,0,14,4,현대건설
3,2,2017-10-15,현대건설,0,1,102,102,198,77,7,...,33,30,3,4.00,100.01,17,0,29,5,KGC인삼공사
4,3,2017-10-17,GS칼텍스,0,1,138,88,175,72,9,...,28,25,0,1.45,100.00,6,0,25,0,한국도로공사
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
943,472,2022-02-26,흥국생명,0,0,111,38,108,32,8,...,16,18,2,0.99,99.99,3,0,15,1,IBK기업은행
944,473,2022-02-27,페퍼저축은행,0,0,116,44,123,36,10,...,16,13,0,1.33,100.01,3,0,12,6,한국도로공사
945,473,2022-02-27,한국도로공사,1,1,124,63,109,49,4,...,21,19,0,3.33,100.01,9,0,11,2,페퍼저축은행
946,474,2022-02-28,GS칼텍스,1,1,164,67,91,52,8,...,21,17,1,3.34,99.98,8,0,7,0,KGC인삼공사


In [117]:
경기별_수치_홈어웨이.to_csv('여자배구_경기별_수치_.csv',index=False)

In [118]:
pd.read_csv('여자배구_경기별_수치_.csv',encoding='utf8')

,경기번호,경기날짜,팀명,홈_어웨이,결과,No.,득점_득점,공격종합_시도,공격종합_성공,공격종합_공격차단,...,블로킹_유효블락,블로킹_실패,블로킹_범실,블로킹_세트당,블로킹_점유율,블로킹_어시스트,벌칙_벌칙,범실_범실,팀코드,상대팀
0,1,2017-10-14,IBK기업은행,1,0,89,69,179,51,15,...,37,30,2,2.25,100.01,8,0,25,3,흥국생명
1,1,2017-10-14,흥국생명,0,1,127,83,193,63,11,...,38,12,0,3.30,100.02,10,0,27,1,IBK기업은행
2,2,2017-10-15,KGC인삼공사,1,0,94,79,205,67,20,...,31,22,2,1.40,99.99,5,0,14,4,현대건설
3,2,2017-10-15,현대건설,0,1,102,102,198,77,7,...,33,30,3,4.00,100.01,17,0,29,5,KGC인삼공사
4,3,2017-10-17,GS칼텍스,0,1,138,88,175,72,9,...,28,25,0,1.45,100.00,6,0,25,0,한국도로공사
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
943,472,2022-02-26,흥국생명,0,0,111,38,108,32,8,...,16,18,2,0.99,99.99,3,0,15,1,IBK기업은행
944,473,2022-02-27,페퍼저축은행,0,0,116,44,123,36,10,...,16,13,0,1.33,100.01,3,0,12,6,한국도로공사
945,473,2022-02-27,한국도로공사,1,1,124,63,109,49,4,...,21,19,0,3.33,100.01,9,0,11,2,페퍼저축은행
946,474,2022-02-28,GS칼텍스,1,1,164,67,91,52,8,...,21,17,1,3.34,99.98,8,0,7,0,KGC인삼공사


In [4]:
IBK = pd.read_csv('직전다섯경기평균.csv',encoding='utf8')
IBK[IBK['팀명']=='IBK기업은행']

,Unnamed: 0,index,팀명,범실_평균_다섯,디그_평균_다섯,리시브_평균_다섯,공격종합_평균_다섯,블로킹_평균_다섯,세트_평균_다섯,서브_평균_다섯
0,0,0,IBK기업은행,19.26755,0.830013,0.378954,0.373139,0.124318,0.347192,0.049483
6,1,6,IBK기업은행,19.26755,0.830013,0.378954,0.373139,0.124318,0.347192,0.049483
10,2,10,IBK기업은행,19.26755,0.830013,0.378954,0.373139,0.124318,0.347192,0.049483
21,3,21,IBK기업은행,19.26755,0.830013,0.378954,0.373139,0.124318,0.347192,0.049483
28,4,28,IBK기업은행,19.26755,0.830013,0.378954,0.373139,0.124318,0.347192,0.049483
...,...,...,...,...,...,...,...,...,...,...
916,151,916,IBK기업은행,19.80000,0.852000,0.306000,0.374000,0.148000,0.348000,0.036000
924,152,924,IBK기업은행,20.80000,0.862000,0.336000,0.386000,0.146000,0.362000,0.034000
929,153,929,IBK기업은행,18.40000,0.844000,0.378000,0.410000,0.160000,0.382000,0.032000
934,154,934,IBK기업은행,17.20000,0.834000,0.392000,0.434000,0.180000,0.408000,0.034000


In [9]:
남자배구 = pd.read_csv('남자배구_경기별_수치.csv')
남자배구.reset_index(inplace=True)
home_index = 남자배구[남자배구['index']/2+1==남자배구['경기번호']].index
away_index = 남자배구[남자배구['index']/2+1!=남자배구['경기번호']].index
남자배구['상대팀'] = 남자배구['팀명']
남자배구.loc[home_index,'상대팀']=남자배구.loc[away_index,'팀명'].values
남자배구.loc[away_index,'상대팀']=남자배구.loc[home_index,'팀명'].values
남자배구.to_csv('남자배구_경기별_수치.csv',index=False)

In [12]:
남자배구 = pd.read_csv('남자배구_경기별_수치.csv')
남자배구

,index,경기번호,경기날짜,팀명,홈_어웨이,결과,득점_득점,공격종합_시도,공격종합_성공,공격종합_공격차단,...,득점점유율,스타팅멤버,팀코드,디그_성공률,세트_성공률,리시브_정확도,블로킹_성공률,총_점수,실점,상대팀
0,0,1,2017-10-14,대한항공,0,0,65,105,55,10,...,1.0,6,0,0.72,0.52,0.56,0.10,100,104,현대캐피탈
1,1,1,2017-10-14,현대캐피탈,1,1,68,94,54,4,...,1.0,6,1,0.70,0.51,0.49,0.21,104,100,대한항공
2,2,2,2017-10-15,KB손해보험,1,1,76,116,61,14,...,1.0,6,2,0.80,0.52,0.48,0.10,106,103,삼성화재
3,3,2,2017-10-15,삼성화재,0,0,68,90,51,5,...,1.0,6,3,0.69,0.54,0.41,0.22,103,106,KB손해보험
4,4,3,2017-10-17,OK저축은행,1,1,78,105,57,5,...,1.0,6,4,0.77,0.51,0.52,0.24,105,101,한국전력
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1215,1215,608,2022-02-11,우리카드,0,0,54,80,45,4,...,1.0,6,6,0.74,0.52,0.26,0.10,85,94,삼성화재
1216,1216,609,2022-02-12,KB손해보험,0,0,78,125,65,12,...,1.0,6,2,0.72,0.51,0.51,0.13,104,106,한국전력
1217,1217,609,2022-02-12,한국전력,1,1,86,123,68,8,...,1.0,6,5,0.67,0.52,0.37,0.19,106,104,KB손해보험
1218,1218,610,2022-02-13,OK금융그룹,0,1,51,82,41,6,...,1.0,6,7,0.89,0.48,0.46,0.15,79,70,현대캐피탈
